# Table of contents 
- [Table of contents](#table-of-contents)
- [Check if a path exists](#check-if-a-path-exists)
- [Trigger a signal within a method](#trigger-a-signal-within-a-method)
- [Functionality to make the users to change their password every 30 days](#functionality-to-make-the-users-to-change-their-password-every-30-days)

# Check if a path exists

In [ ]:
import os

def dir_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Trigger a signal within a method

In [ ]:
# signals.py
from django.dispatch import Signal, receiver
from Services import ProcessSignal

signal_to_trigger = Signal()

@receiver(signal_to_trigger)
def handle_signal_to_trigger(sender, item):
    service = ProcessSignal()
    service.process_signal(sender, item)

In [ ]:
# services.py 
class ProcessSignal:
    def process_signal(self, sender, item):
        print(f"Processing signal from {sender.__class__.__name__} with item {item}")

In [ ]:
# views.py
from django.http import request
from signals import signal_to_trigger

def view_that_triggers_signal(request):
    print("View that triggers the signal")
    signal_to_trigger.send(
        sender=request.user,
        item="The model or data"
    )

# Functionality to make the users to change their password every 30 days

In [ ]:
# models.py
from django.db import models
from django.contrib.auth.models import User
from django.utils import timezone

# Create your models here.
class PasswordHistory(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE)
    password_change_date = models.DateTimeField(default=timezone.now)
    password_next_change_date = models.DateTimeField(default=timezone.now)
    
    class Meta:
        get_latest_by = 'password_change_date'
        
    def is_change_required(self):
        return timezone.now() >= self.password_next_change_date
        
    def __str__(self):
        return f"Last password change {self.password_change_date} for user {self.user.first_name}"

In [ ]:
# signals.py
"""Signals for the user's password tracking"""
from django.contrib.auth.models import User
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.contrib.auth.signals import user_logged_in 
from .models import PasswordHistory
from django.utils import timezone
from datetime import timedelta 

@receiver(post_save, sender=User)
def create_password_schedule(sender, instance, created, **kwargs):
    if created:
        PasswordHistory.objects.create(
            user=instance,
            password_change_date=timezone.now(),
            password_next_change_date=timezone.now()
        )
        
@receiver(user_logged_in)
def force_password_change_check(sender, request, user, **kwargs):
    schedule, created = PasswordHistory.objects.get_or_create(
        user=user,
        defaults={
            'password_change_date': timezone.now(),
            'password_next_change_date': timezone.now() 
        }
    )
    if created or schedule.is_change_required():
        request.session['password_change_required'] = True
    else:
        request.session['password_change_required'] = False
    request.session.modified = True # Force save the session

In [ ]:
# middlewares.py
from django.shortcuts import redirect
from django.urls import reverse
import logging

logger = logging.getLogger(__name__)

class PasswordExpiryMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        # Add logging to verify the middleware is running
        if request.user.is_authenticated:
            change_required = request.session.get('password_change_required', False)
            logger.debug(f"User {request.user.username} authenticated, password_change_required: {change_required}")
            
            if change_required:
                change_password_url = reverse('password_change')
                logout_url = reverse('logoutuser')
                
                # Don't redirect if already on these pages
                if request.path != change_password_url and request.path != logout_url:
                    logger.debug(f"Redirecting user {request.user.username} to password change")
                    return redirect(change_password_url)
        
        return self.get_response(request)

In [ ]:
# views.py
from django.contrib.auth.views import PasswordChangeView
from django.urls import reverse_lazy
from .forms import SchedulePasswordChangeForm

# Create your views here.
class ScheduledPasswordChangeView(PasswordChangeView):
    """Password change view handler for the user's password
    
    Args:
        PasswordChangeView: Django's built-in password change view
    """
    form_class = SchedulePasswordChangeForm
    success_url = reverse_lazy('index')
    template_name = 'password_change.html'
    
    def get_form_kwargs(self):
        kwargs = super().get_form_kwargs()
        kwargs['request'] = self.request
        return kwargs
    


In [ ]:
# forms.py
from django.contrib.auth.forms import PasswordChangeForm
from django.utils import timezone
from django.core.exceptions import ValidationError
from datetime import timedelta
from .models import PasswordHistory

class SchedulePasswordChangeForm(PasswordChangeForm):
    def __init__(self, user=None, request=None, *args, **kwargs):
        self.request = request
        super().__init__(user, *args, **kwargs)
        
    def save(self, commit=True):
        user = super().save(commit)
        now = timezone.now()
        next_change = now + timedelta(days=30)
        schedule, created = PasswordHistory.objects.get_or_create(
            user=user, 
            defaults={
                'password_change_date': now,
                'password_next_change_date': next_change,
            }
        )
        if not created:
            schedule.password_change_date = now
            schedule.password_next_change_date = next_change
            schedule.save()
            
        if self.request:
            self.request.session['password_change_required'] = False
            
        return user

In [ ]:
# settings.py
MIDDLEWARE = [
    'app.middleware.PasswordExpiryMiddleware',
]

In [ ]:
# apps.py
# In your apps.py
from django.apps import AppConfig

class YourAppConfig(AppConfig):
    name = 'your_app'

    def ready(self):
        import your_app.signals  # Import the signals module

In [ ]:
{% extends "layout.html" %}
{% load static %}
<link href="{% static "css/password_change.css" %}" rel="stylesheet" >

{% block content %}

<div class="row align-items-center justify-content-center vh-100">
    <div class="col-md-5 col-lg-4 d-flex flex-column py-6">
        <img src="{% static "image.png" %}">
    </div>
    <div class="col-md-5 col-lg-4 d-flex flex-column py-6">
        <h1 class="mb-2 text-center title">Re-establish your password</h1>
        <p class="text-center secondary-text">For safety reasons, you must change your password again to access the techtool. This will be requested again in one month.</p>
        <form method="POST">
            {% csrf_token %}                
            <div class="mb-4">
                <label for="id_old_password">Current Password</label>
                <input type="password" 
                        name="old_password" 
                        id="id_old_password" 
                        class="form-control {% if form.old_password.errors %}is-invalid{% endif %}"
                        required>
                {% for error in form.old_password.errors %}
                    <div class="invalid-feedback">{{ error }}</div>
                {% endfor %}
            </div>
            <div class="mb-4">
                <label for="id_new_password1">New Password</label>
                <input type="password" 
                        name="new_password1" 
                        id="id_new_password1" 
                        class="form-control {% if form.new_password1.errors %}is-invalid{% endif %}"
                        required>
                {% for error in form.new_password1.errors %}
                    <div class="invalid-feedback">{{ error }}</div>
                {% endfor %}
            </div>
            <div class="mb-4">
                <label for="id_new_password2">Confirm New Password</label>
                <input type="password" 
                        name="new_password2" 
                        id="id_new_password2" 
                        class="form-control {% if form.new_password2.errors %}is-invalid{% endif %}"
                        required>
                {% for error in form.new_password2.errors %}
                    <div class="invalid-feedback">{{ error }}</div>
                {% endfor %}
            </div>

            <button type="submit" class="button-primary w-100 mt-3 mb-2">Change password</button>
        </form>
    </div>
</div>

{% endblock content %}